## Your Personal Career Agent
---
    powered with Resources

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

c:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
# Loading LinkedIn data

reader = PdfReader("me/Linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:

print(linkedin)

   
Contact
ksharma9719@gmail.com
www.linkedin.com/in/jai-keshav-
sharma (LinkedIn)
Top Skills
Amazon Web Services (AWS)
Cloud
Cloud Applications
Jai Keshav Sharma
Student at Chhattisgarh Swami Vivekanand Technical University
(CSVTU)
Bhilai, Chhattisgarh, India
Summary
President (MATLAB Club) & Outreach Coordinator | Programmers'
Paradise Technical Society
Skills & Interests
Programming Languages: C, C++, Python, R
Interests: Data Analysis, Artificial Intelligence, Machine Learning,
Competitive Programming, Visualization, Statistical Modeling
Software Development: Version Control (Git)
Interpersonal Skills: Team Collaboration, Public Speaking,
Leadership
Experience
Programmers' Paradise
President (MATLAB Club)
August 2024 - Present (1 year)
Bhilai, Chhattisgarh, India
TECHPLEMENT
Cloud AWS Intern
May 2025 - June 2025 (2 months)
Contributing as a Cloud AWS Intern by supporting the design, deployment,
and management of cloud-based solutions. Gaining hands-on experience
across various AWS

In [5]:
# Loading summary

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
print(summary)

🔭 I’m currently working on:
Building innovative machine learning models for social good and advancing my programming skills.

🤝 I’m looking to collaborate on:
Hackathons, impactful projects in AI, machine learning, and software development.

🛠️ I’m looking for help with:
Exploring advanced AI research and enhancing my expertise in machine learning and software engineering.

🌱 I’m currently learning:
Cutting-edge topics in Machine Learning, Data Structures and Algorithms, Object-Oriented Programming (C++), and Database Management Systems.

💬 Ask me about:
Building predictive models, hosting tech events, and gaming strategies for tactical and cooperative games.

⚡ Fun fact:
I’m not only an AI enthusiast but also a problem-solver on platforms like LeetCode and Code360, blending coding with gaming to keep life interesting!


### ChatBot on behalf of the User

In [7]:
name = "Jai Keshav Sharma"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [9]:

system_prompt

"You are acting as Jai Keshav Sharma. You are answering questions on Jai Keshav Sharma's website, particularly questions related to Jai Keshav Sharma's career, background, skills and experience. Your responsibility is to represent Jai Keshav Sharma for interactions on the website as faithfully as possible. You are given a summary of Jai Keshav Sharma's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n🔭 I’m currently working on:\nBuilding innovative machine learning models for social good and advancing my programming skills.\n\n🤝 I’m looking to collaborate on:\nHackathons, impactful projects in AI, machine learning, and software development.\n\n🛠️ I’m looking for help with:\nExploring advanced AI research and enhancing my expertise in machine learning and software engineering.\n\n🌱 I’m currently lear

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [11]:

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


### ChatBot with an Evaluator 

In [12]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY") 
)

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gpt-4.1-nano", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):

    # forcefully introducing unacceptable reply
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is gibberish, using Pig Latin instead of clear, professional language. It does not appropriately address the user's question about having a patent and is unprofessional. The response fails to provide any relevant information or engage constructively with the user's query.
